In [1]:
import networkx as nx

from regraph import Rule, plot_rule
from regraph.neo4j.hierarchy import Neo4jHierarchy
import regraph.neo4j.cypher_utils as cypher

## Initializing Neo4j database

0. When installing neo4j you will be asked to choose login/password for you dbs (here its "neo4j"/"admin"), if you choose other -- change in the cell below.
1. To start neo4j server run `sudo service neo4j start`
2. Check status by running `sudo service neo4j status`. Here you can check the _bolt_ port, change the cell below if different from 7687
3. You can query the db by using the neo4j browser, the address can be found also in the result of 'status', e.g. "Remote interface available at http://localhost:7474/".

In [2]:
# initialize the neo4j driver, wrappped into Neo4jGraph object
h = Neo4jHierarchy(uri="bolt://localhost:7687", user="neo4j", password="admin")
h._clear()

## Adding a graph to the hierarchy

In [3]:
nodes = [
    ("a", {"shape": ["circle"], "color": ["white"]}),
    ("b", {"shape": ["circle"], "color": ["black"]}),
    ("d", {"shape": ["circle"], "color": ["grey"]}),
    ("c", {"shape": ["square"], "color": ["white", "black"]})
]
edges = [
    ("a", "b"),
    ("b", "c"),
    ("b","d")
]
h.add_graph('graphB',nodes, edges)

In [4]:
nodes = [
    ("a", {"shape": ["square"], "color": ["white"]}),
    ("b", {"shape": ["square"], "color": ["black"]}),
    ("c", {"shape": ["circle"], "color": ["white", "black"]}),
    ("d", {"shape": ["triangle"], "color":["grey"]})
]
edges = [
    ("a", "b"),
    ("c", "b"),
    ("c", "a"),
    ("c", "c"),
    ("c", "d")
]
h.add_graph('graphC', nodes, edges)

In [5]:
nodes = [
    ("a", {"shape": ["circle"], "color": ["white", "black", "grey"]}),
    ("b", {"shape": ["square"], "color": ["white", "black"]}),
    ("c", {"shape": ["triangle"], "color":["grey"]})
]
edges = [
    ("a", "b"),
    ("a", "a"),
    ("b", "b"),
    ("a", "c")
]
h.add_graph('graphD', nodes, edges)

## Typing

In [6]:
mappingBD = {
    "a":"a",
    "b":"a",
    "c":"b",
    "d":"a"
}
h.add_typing('graphB', 'graphD', mappingBD)
mappingCD = {
    "a":"b",
    "b":"b",
    "c":"a",
    "d":"c"
}
h.add_typing('graphC', 'graphD', mappingCD)

// match all typing pairs between 'graphB' and 'graphD'
MATCH (s:graphB), (t:graphD)
WITH s, t
// match all the predecessors of 's' and successors of 't'
MATCH (pred), (suc)
WHERE (pred)-[:typing*0..]->(s)
	AND (t)-[:typing*0..]->(suc)
WITH s, t, collect(DISTINCT pred) as pred_list, collect(DISTINCT suc) as suc_list
// select all the pairs 'pred' 'suc' with a path between
UNWIND pred_list as pred
UNWIND suc_list as suc
OPTIONAL MATCH (pred)-[r:typing*]->(suc)
WHERE NONE(rel in r WHERE rel.tmp = 'True')
WITH s, t, r, labels(pred)[1] as pred_label, labels(suc)[1] as suc_label
WHERE r IS NOT NULL
WITH DISTINCT s, t, pred_label, suc_label
// return the pairs 's' 't' where there should be a typing edge
OPTIONAL MATCH (s)-[new_typing:typing]->(t)
WHERE new_typing.tmp IS NOT NULL
WITH pred_label, suc_label, s.id as s_id, t.id as t_id, new_typing
WHERE new_typing IS NULL
RETURN pred_label, suc_label, s_id, t_id

// match all typing pairs between 'graphC' and 'graphD'
MATCH (s:graphC), (t:graph

## Pullback

In [7]:
cypher.pullback('graphB', 'graphC', 'graphD', 'graphA')

("MATCH (n:graphB)-[:typing]->(:graphD)<-[:typing]-(m:graphC)\n\n//Perform the intersection of the properties of n, m\nWITH [] as new_props, m, n\nWITH new_props + REDUCE(pairs = [], k in keys(n) | \n\tCASE WHEN ALL(others in [m] WHERE k in keys(others))\n\tTHEN\n\t\tpairs + REDUCE(inner_pairs = [], v in n[k] | \n\t\t\tCASE WHEN ALL(others in [m] WHERE v in others[k])\n\t\t\tTHEN\n\t\t\t\tinner_pairs + {key: k, value: v}\n\t\t\tELSE\n\t\t\t\tinner_pairs\n\t\t\tEND)\n\tELSE\n\t\tpairs\n\tEND) as new_props, m, n\nWITH apoc.map.groupByMulti(new_props, 'key') as new_props, m, n\nWITH apoc.map.fromValues(REDUCE(pairs=[], k in keys(new_props) | \n\tpairs + [k, REDUCE(values=[], v in new_props[k] | \n\t\tvalues + CASE WHEN v.value IN values THEN [] ELSE v.value END)])) as new_props, m, n\nCREATE (new_node_a:graphA) \nSET new_node_a.id = toString(id(new_node_a)) \nWITH toString(id(new_node_a)) as id_var , m, new_props, new_node_a, n\nSET new_node_a += new_props\nSET new_node_a.id = toString(id

## Pushout

In [8]:
h.remove_node('graphD')

In [9]:
cypher.pushout('graphA', 'graphB', 'graphC', 'graphD')

('\n// We copy the nodes of B in D\nMATCH (n:node:graphB)\n// create a node corresponding to the clone\nCREATE (m:node:graphD) \nMERGE (n)-[:typing]->(m)WITH m, toString(id(m)) as m_id_var, n.id as original_old, n \n// set the id property of the original node to NULL\nSET n.id = NULL\n// copy all the properties of the original node to the clone\nSET m = n\n// set id property of the clone to neo4j-generated id\nSET m.id = toString(id(m)), m.count = NULL\n// set back the id property of the original node\nSET n.id = original_old\nWITH m, toString(id(m)) as m_id_var, n \nWITH [] as sucIgnore, [] as predIgnore, m, n \n// Match successors and out-edges of a node to be cloned in the clone graph\nOPTIONAL MATCH (n)-[out_edge:edge]->(:node:graphB)-[:typing]->(suc:node:graphD)\nWHERE NOT suc.id IS NULL AND NOT suc.id IN sucIgnore\nWITH collect({neighbor: suc, edge: out_edge}) as suc_maps, predIgnore, m, n \n// match predecessors and in-edges of a node to be cloned in the clone graph\nOPTIONAL MA

In [10]:
h.remove_node('graphB', reconnect=True)

In [11]:
h._clear()
h._drop_all_constraints()
h.close()